In [74]:
! pip install replicate requests aiohttp python-dotenv Pillow pyfs

^C


In [84]:
import asyncio
from collections import defaultdict
import json
import aiohttp
import requests
import os
from dotenv import load_dotenv

load_dotenv()

last_fm_api_base_url = "http://ws.audioscrobbler.com/2.0/"


def get_top_200_last_fm_artists():
    top_200_last_fm_artists = []
    for page_number in range(1, 5):
        fifty_top_artists = requests.get(last_fm_api_base_url + "?method=chart.gettopartists&api_key=" + os.environ["LAST_FM_API_KEY"] + "&format=json&page=" + str(page_number)).json()
        for artist in fifty_top_artists["artists"]["artist"]:
            top_200_last_fm_artists.append(artist["name"])
    return top_200_last_fm_artists



def get_top_three_albums_for_all_artists(artists):
    top_three_albums = defaultdict(list)
    for artist in artists:
        top_three_albums[artist] = [album["name"] for album in requests.get(last_fm_api_base_url + "?method=artist.gettopalbums&artist=" + artist + "&api_key=" + os.environ["LAST_FM_API_KEY"] + "&format=json&limit=3").json()["topalbums"]["album"]]
    return top_three_albums


def generate_urls_to_get_top_three_albums(top_three_albums_for_artists):
    urls = []
    for artist, albums in top_three_albums_for_artists.items():
        for album in albums:
            urls.append(last_fm_api_base_url + "?method=album.getinfo&api_key=" + os.environ["LAST_FM_API_KEY"] + "&artist=" + artist + "&album=" + album + "&format=json")
    return urls


urls = generate_urls_to_get_top_three_albums(get_top_three_albums_for_all_artists(get_top_200_last_fm_artists()))

json.dump(urls, open("urls.json", "w+"), indent = 4, ensure_ascii=False)

In [ ]:
async def get_album_with_info(session, url):
    async with session.get(url) as response:
        return await response.json()


async def get_albums_with_info(urls):
    async with aiohttp.ClientSession() as session:
        albums_with_info = [get_album_with_info(session, url) for url in urls]
        return await asyncio.gather(*albums_with_info)




In [85]:
import json
import replicate
from PIL import Image
import fs
from io import BytesIO


os.environ["REPLICATE_API_TOKEN"]
input = {
    "image": "https://th.bing.com/th/id/OIP.YSla4CEVSKJWVUR1c6_QawAAAA?rs=1&pid=ImgDetMain",
    "mask_limit": -1,
    "mask_2_mask": False, 
    "crop_n_layers": 0,
    # "box_nms_thresh": 0.5,
    "points_per_side": 64,
    # "pred_iou_thresh": 0.5,
    # "multimask_output": 'True',
    # "points_per_batch": 1000,
    # "min_mask_region_area": 0,
    # "stability_score_offset": 0.1,
    "stability_score_thresh": 0.5,
    # "crop_n_points_downscale_factor": 1.5,
}

output = replicate.run(
    "lucataco/segment-anything-2:be7cbde9fdf0eecdc8b20ffec9dd0d1cfeace0832d4d0b58a071d993182e1be0",
    input=input
)

output_response = requests.get(output[0])
mask = Image.open(BytesIO(output_response.content))

input_response = requests.get(input.get("image"))
input_image = Image.open(BytesIO(input_response.content))

printOutput = Image.alpha_composite(input_image, mask)
printOutput.show()
print('hi', input.get("image"))
print(json.dumps(output))



MissingSchema: Invalid URL 'h': No scheme supplied. Perhaps you meant https://h?